## Time Series Evaluation (Multiple Datasets)


In [1]:
# Install automl_tool in editable mode
!pip install -e .


Obtaining file:///Users/430016232/Library/CloudStorage/OneDrive-EnactMortgageInsuranceCompany/Documents/research/automl_tool
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for automl_tool (pyproject.toml) ... done
  Created wheel for automl_tool: filename=automl_tool-0.1.0-0.editable-py3-none-any.whl size=2921 sha256=1488d7a54dad47129b8f17819f6a069b74c871dd4974af16434c27f2039b8d75
  Stored in directory: /private/var/folders/sz/43lqpbtj4n3dyzfj6zq2ljgw0000gq/T/pip-ephem-wheel-cache-4jh7y36o/wheels/c6/37/80/5eb8f2f50997eaa77b7c4671e4ff5decdee87e31c513e5fe4a
Successfully built automl_tool
  Attempting uninstall: automl_tool
    Found existing installation: automl_tool 0.1.0
    Uninstalling automl_tool-0.1.0:
      Successfully uninstalled automl_tool-0.1.0


### Get the datasets

In [2]:
import numpy as np
import pandas as pd
import warnings
from sklearn.metrics import mean_absolute_error
from automl_tool.automl import AutoML
from automl_tool.preprocessing import ts_train_test_split
from pandas.tseries.frequencies import to_offset

warnings.filterwarnings("ignore")
np.random.seed(42)

# Helpers
make_dates = lambda n, freq='ME', start='2000-01-01': pd.date_range(start, periods=n, freq=freq)

def df_from_values(values, start='2000-01-01', freq='ME'):
    return pd.DataFrame({'date': make_dates(len(values), freq, start), 'value': np.asarray(values, dtype=float)})

def ensure_regular_frequency(df: pd.DataFrame, date_col: str = 'date', value_col: str = 'value') -> pd.DataFrame:
    d = df.copy()
    d[date_col] = pd.to_datetime(d[date_col])
    d = d.sort_values(date_col).drop_duplicates(subset=[date_col])
    # Try to infer frequency; fallback to median delta if needed
    freq = pd.infer_freq(d[date_col])
    if freq is None:
        deltas = d[date_col].diff().dropna()
        if len(deltas) == 0:
            freq = 'D'
        else:
            td = deltas.median()
            try:
                freq = to_offset(td)
            except Exception:
                freq = 'D'
    full_index = pd.date_range(d[date_col].iloc[0], d[date_col].iloc[-1], freq=freq)
    d = d.set_index(date_col).reindex(full_index)
    # Interpolate missing values over time; fallback to ffill then bfill
    if d[value_col].isna().any():
        try:
            d[value_col] = d[value_col].interpolate(method='time')
        except Exception:
            d[value_col] = d[value_col].ffill().bfill()
    d = d.reset_index().rename(columns={'index': date_col})
    return d[[date_col, value_col]]

# Synthetic dataset generators (diverse shapes)
def synth_seasonal(n=1000):
    t = np.arange(n)
    y = 10*np.sin(2*np.pi*t/12) + 0.5*t + np.random.normal(0, 2, n)
    return df_from_values(y)

def synth_linear(n=1000):
    t = np.arange(n)
    y = 0.3*t + np.random.normal(0, 3, n)
    return df_from_values(y)

def synth_quadratic(n=1000):
    t = np.arange(n)
    y = 0.01*(t-n/2)**2 + np.random.normal(0, 2, n)
    return df_from_values(y)

def synth_logistic(n=1000):
    t = np.arange(n)
    midpoint = n/2
    y = 100/(1+np.exp(-(t-midpoint)/10)) + np.random.normal(0, 2, n)
    return df_from_values(y)

def synth_walk(n=1000):
    rng = np.random.default_rng(123)
    y = np.cumsum(rng.normal(0.3, 1.0, n))
    return df_from_values(y)

def synth_piecewise(n=1000):
    t = np.arange(n)
    k1 = int(n*0.3); k2 = int(n*0.65)
    base = np.piecewise(t,
                        [t < k1, (t >= k1) & (t < k2), t >= k2],
                        [lambda x: 0.2*x,
                         lambda x: (0.2*k1) + (-0.1)*(x-k1),
                         lambda x: (0.2*k1) + (-0.1)*(k2-k1) + 0.4*(x-k2)])
    y = base + np.random.normal(0, 2, n)
    return df_from_values(y)

def synth_spiky(n=1000):
    rng = np.random.default_rng(0)
    y = 20 + np.sin(2*np.pi*np.arange(n)/24) + rng.normal(0, 2, n)
    idx = rng.choice(n, size=max(10, n//30), replace=False)
    y[idx] += rng.uniform(10, 25, len(idx))
    return df_from_values(y)

def synth_multiseason(n=1000):
    t = np.arange(n)
    y = 5*np.sin(2*np.pi*t/12) + 3*np.sin(2*np.pi*t/6) + np.random.normal(0, 2, n)
    return df_from_values(y)

# Collect datasets (n=1000 each synthetic)
datasets = {
    'Seasonal+Trend': synth_seasonal(n=1000),
    'Linear Trend': synth_linear(n=1000),
    'Quadratic': synth_quadratic(n=1000),
    'Logistic (S-curve)': synth_logistic(n=1000),
    'Random Walk (drift)': synth_walk(n=1000),
    'Piecewise (changepoints)': synth_piecewise(n=1000),
    'Spiky Intermittent': synth_spiky(n=1000),
    'Multi-seasonal': synth_multiseason(n=1000),
}

# Add real datasets from statsmodels
try:
    import statsmodels.api as sm
    # Sunspots (yearly)
    try:
        sun = sm.datasets.sunspots.load_pandas().data
        df_sun = pd.DataFrame({
            'date': pd.to_datetime(sun['YEAR'], format='%Y', errors='coerce'),
            'value': sun['SUNACTIVITY'].astype(float)
        }).dropna()
        datasets['Sunspots'] = df_sun
    except Exception:
        pass

    # Mauna Loa CO2 (weekly)
    try:
        co2 = sm.datasets.co2.load_pandas().data
        co2 = co2.copy()
        if 'co2' in co2.columns and 'date' not in co2.columns:
            co2 = co2.reset_index().rename(columns={'index': 'date', 'co2': 'value'})
        else:
            if 'date' not in co2.columns:
                co2 = co2.reset_index().rename(columns={'index': 'date'})
            if 'value' not in co2.columns:
                first_val = [c for c in co2.columns if c != 'date'][0]
                co2 = co2.rename(columns={first_val: 'value'})
        co2 = co2[['date', 'value']].dropna()
        datasets['CO2'] = co2
    except Exception:
        pass
except Exception:
    pass

# Add selected FRED series
try:
    from pandas_datareader import data as pdr
    fred_codes = ['CPIAUCSL', 'UNRATE', 'INDPRO']
    for code in fred_codes:
        try:
            df_fred = pdr.DataReader(code, 'fred', start='1990-01-01')
            df_fred = df_fred.rename(columns={code: 'value'}).reset_index().rename(columns={'DATE': 'date'})
            df_fred = df_fred.dropna()
            datasets[code] = df_fred
        except Exception:
            continue
except Exception:
    pass

# Filter to only the datasets the user wants
allowed = {
    'Seasonal+Trend', 'Linear Trend', 'Quadratic', 'Logistic (S-curve)',
    'Random Walk (drift)', 'Piecewise (changepoints)', 'Spiky Intermittent',
    'Multi-seasonal', 'Sunspots', 'CO2', 'CPIAUCSL', 'UNRATE', 'INDPRO'
}
datasets = {k: v for k, v in datasets.items() if k in allowed}


### AutoML

In [3]:

winners = []
# Normalize to equidistant dates
for k in list(datasets.keys()):
    datasets[k] = ensure_regular_frequency(datasets[k], 'date', 'value')

fdw, holdout_window, forecast_window = 18, 24, 1
maes = []

for name, df in datasets.items():
    if len(df) < fdw + holdout_window + forecast_window + 1:
        continue
    X, y = df, df['value']
    X_train, X_holdout, y_train, y_holdout = ts_train_test_split(
        X, y, 'value', 'date', fdw, holdout_window, forecast_window=forecast_window
)
    # Skip datasets that are too short for 5-fold TimeSeriesSplit with this test size
    if len(X_train) <= 5 * holdout_window:
        continue
    automl_mod = AutoML(X_train, y_train, 'value', time_series=True)
    automl_mod.fit_pipeline(holdout_window=holdout_window)
    preds = automl_mod.fitted_pipeline.best_estimator_.predict(X_holdout)
    mae = mean_absolute_error(y_holdout, preds)
    maes.append(mae)
    print(f"{name}: {mae:.3f}")
	# ... inside your datasets loop, after fit:
    winners.append(type(automl_mod.fitted_pipeline.best_estimator_.get_params()['model']).__name__)

if maes:
    print(f"Average MAE: {float(np.mean(maes)):.3f}")

Seasonal+Trend: 1.861
Linear Trend: 2.304
Quadratic: 1.872
Logistic (S-curve): 2.419
Random Walk (drift): 0.708
Piecewise (changepoints): 2.421
Spiky Intermittent: 1.739
Multi-seasonal: 1.481
Sunspots: 9.909
CO2: 0.216
CPIAUCSL: 0.402
UNRATE: 0.112
INDPRO: 0.484
Average MAE: 1.994


In [7]:
((1.994 * 13) - 9.909)/12

1.3344166666666666

In [6]:
# ((2.493 * 10) - 9.909)/9

In [12]:
winners

['ElasticNet',
 'ElasticNet',
 'ElasticNet',
 'ElasticNet',
 'ElasticNet',
 'ElasticNet',
 'ElasticNet',
 'XGBWithEarlyStoppingRegressor',
 'ElasticNet',
 'ElasticNet',
 'ElasticNet',
 'ElasticNet',
 'ElasticNet']

### Prophet Performance

In [125]:
# %pip install plotly

In [126]:
import logging
logger = logging.getLogger('cmdstanpy')
logger.handlers = []
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

import numpy as np
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from itertools import product

# Enhanced Prophet benchmarking block (with yearly-series safeguards + expanded hyperparameter grid)
# Configuration
MIN_TRAIN_POINTS = 30               # skip very short series
LOG_TOL = 1e-6                      # small constant for log transform
OUTLIER_Z = 4.0                     # z-score threshold for winsorization
HYPERPARAM_GRID = {
    'changepoint_prior_scale': [.005, 0.05, 0.5],
    'seasonality_prior_scale': [5.0, 15.0],
    'yearly_fourier': [10, 20],
    'changepoint_range': [0.8, 0.9],
    'seasonality_mode': ['additive', 'multiplicative'],
    'growth': ['linear', 'logistic']  # logistic will auto-add cap/floor
}
ENABLE_MONTHLY = True
ENABLE_QUARTERLY = True
USE_LOG = True                      # per-series decision still checked
ROBUST_TREND = True                 # remove extreme spikes before fit
LOGISTIC_CAP_MULT = 1.15            # cap = max(train_y) * this
LOGISTIC_FLOOR_MULT = 0.85          # floor = min(train_y) * this (if positive)

prophet_maes = {}
prophet_mapes = {}
prophet_details = {}

# Helper functions

def _prepare_series(df: pd.DataFrame) -> pd.DataFrame:
    s = (df[['date','value']]
          .rename(columns={'date':'ds','value':'y'})
          .sort_values('ds')
          .reset_index(drop=True))
    s = s.drop_duplicates(subset='ds')
    s['ds'] = pd.to_datetime(s['ds'])
    s = s.dropna(subset=['y'])
    return s

def _winsorize_outliers(y: pd.Series, z=OUTLIER_Z):
    if y.std() == 0:
        return y
    zscores = (y - y.mean()) / y.std()
    clipped = y.copy()
    mask_hi = zscores > z
    mask_lo = zscores < -z
    if mask_hi.any():
        clipped[mask_hi] = y[~mask_hi].max()
    if mask_lo.any():
        clipped[mask_lo] = y[~mask_lo].min()
    return clipped

def _maybe_log_transform(s: pd.DataFrame) -> tuple[pd.DataFrame, bool]:
    if not USE_LOG:
        return s, False
    if (s['y'] <= 0).any():
        return s, False
    y_range = s['y'].max() / max(s['y'].min(), LOG_TOL)
    if y_range > 20:
        s = s.copy()
        s['y'] = np.log(s['y'] + LOG_TOL)
        return s, True
    return s, False

def _inverse_log(yhat: np.ndarray) -> np.ndarray:
    return np.exp(yhat) - LOG_TOL

def _classify_temporal_resolution(train_df: pd.DataFrame) -> str:
    if len(train_df) < 3:
        return 'unknown'
    deltas = np.diff(train_df['ds'].sort_values().values).astype('timedelta64[D]').astype(int)
    med = np.median(deltas)
    if med <= 2: return 'daily'
    if med <= 10: return 'weekly'
    if med <= 40: return 'monthly'
    if med <= 120: return 'quarterly'
    return 'yearly'

# Iterate datasets
for name, df in datasets.items():
    if len(df) < fdw + holdout_window + forecast_window + 1:
        continue

    series = _prepare_series(df)
    if len(series) <= holdout_window + MIN_TRAIN_POINTS:
        continue

    train_end = len(series) - holdout_window
    train_df = series.iloc[:train_end].copy()
    holdout_df = series.iloc[train_end:].copy()

    inferred_freq = pd.infer_freq(train_df['ds'])
    temporal_res = _classify_temporal_resolution(train_df)

    local_enable_monthly = ENABLE_MONTHLY
    local_enable_quarterly = ENABLE_QUARTERLY

    if temporal_res == 'yearly':
        freq = 'YS'
        local_enable_monthly = False
        local_enable_quarterly = False
        effective_holdout = min(holdout_window, 10)
    else:
        freq = inferred_freq or 'MS'
        effective_holdout = holdout_window

    if ROBUST_TREND:
        train_df = train_df.copy()
        train_df['y'] = _winsorize_outliers(train_df['y'], z=OUTLIER_Z)

    train_df, used_log = _maybe_log_transform(train_df)
    if used_log:
        holdout_df_t = holdout_df.copy()
        holdout_df_t['y'] = np.log(holdout_df_t['y'] + LOG_TOL)
    else:
        holdout_df_t = holdout_df

    if temporal_res == 'yearly':
        yearly_orders = [5, 10]
    else:
        yearly_orders = HYPERPARAM_GRID['yearly_fourier']

    cps_list = HYPERPARAM_GRID['changepoint_prior_scale']
    sps_list = HYPERPARAM_GRID['seasonality_prior_scale']
    cpr_list = HYPERPARAM_GRID['changepoint_range']
    smode_list = HYPERPARAM_GRID['seasonality_mode']
    growth_list = HYPERPARAM_GRID['growth']

    best_mae = np.inf
    best_cfg = None
    best_pred = None

    for cps, sps, yf, cpr, smode, growth in product(cps_list, sps_list, yearly_orders, cpr_list, smode_list, growth_list):
        # Skip logistic if we applied log transform (incompatible semantics)
        if used_log and growth == 'logistic':
            continue

        # Prepare working copies for logistic caps/floors
        if growth == 'logistic':
            # Ensure positive range; fallback to linear if degenerate
            y_train_vals = train_df['y'].values
            y_min, y_max = np.min(y_train_vals), np.max(y_train_vals)
            if y_max <= y_min + 1e-9:
                continue
            cap_val = y_max * LOGISTIC_CAP_MULT
            floor_val = y_min * LOGISTIC_FLOOR_MULT if y_min > 0 else 0.0
            train_use = train_df.copy()
            train_use['cap'] = cap_val
            train_use['floor'] = floor_val
        else:
            train_use = train_df

        try:
            m = Prophet(
                yearly_seasonality=False,
                weekly_seasonality=False,
                daily_seasonality=False,
                changepoint_prior_scale=cps,
                seasonality_prior_scale=sps,
                changepoint_range=cpr,
                seasonality_mode=smode,
                growth=growth,
            )
        except Exception:
            continue

        # Add seasonalities
        try:
            m.add_seasonality(name='yearly', period=365.25, fourier_order=yf)
        except Exception:
            pass
        if local_enable_monthly and temporal_res in {'monthly','weekly','daily'}:
            m.add_seasonality(name='monthly', period=30.5, fourier_order=6)
        if local_enable_quarterly and temporal_res in {'monthly','weekly','daily','quarterly'}:
            m.add_seasonality(name='quarterly', period=91.25, fourier_order=4)
        if temporal_res in {'daily','weekly'}:
            # Provide moderate weekly component for higher frequency data
            try:
                m.add_seasonality(name='weekly_custom', period=7, fourier_order=4)
            except Exception:
                pass

        try:
            m.fit(train_use)
        except Exception:
            continue

        try:
            future = m.make_future_dataframe(periods=effective_holdout, freq=freq)
        except OverflowError:
            try:
                future = m.make_future_dataframe(periods=min(3, effective_holdout), freq=freq)
            except Exception:
                continue
        except Exception:
            continue

        # Add cap/floor to future if logistic
        if growth == 'logistic':
            future['cap'] = cap_val
            future['floor'] = floor_val

        try:
            fcst = m.predict(future).set_index('ds')
        except Exception:
            continue

        aligned_holdout = holdout_df.iloc[:effective_holdout]
        needed = aligned_holdout['ds']
        if not set(needed).issubset(fcst.index):
            continue

        yhat = fcst.loc[needed, 'yhat'].values
        actual = aligned_holdout['y'].values
        if used_log:
            yhat = _inverse_log(yhat)
            actual = aligned_holdout['y'].values  # already original scale

        mae = mean_absolute_error(actual, yhat)
        if mae < best_mae:
            best_mae = mae
            best_cfg = {
                'changepoint_prior_scale': cps,
                'seasonality_prior_scale': sps,
                'yearly_fourier_order': yf,
                'changepoint_range': cpr,
                'seasonality_mode': smode,
                'growth': growth,
                'log_transform': used_log,
                'freq': freq,
                'temporal_resolution': temporal_res,
                'effective_holdout': effective_holdout,
            }
            best_pred = yhat

    if best_pred is None:
        print(f"{name}: Prophet failed; skipping.")
        continue

    eval_holdout = holdout_df.iloc[:best_cfg['effective_holdout']]
    mape = mean_absolute_percentage_error(eval_holdout['y'].values, best_pred)
    prophet_maes[name] = best_mae
    prophet_mapes[name] = mape
    prophet_details[name] = {
        'config': best_cfg,
        'mae': best_mae,
        'mape': mape,
        'n_train': len(train_df),
        'n_holdout_used': best_cfg['effective_holdout'],
        'temporal_resolution': temporal_res,
    }

    print(f"{name} (Prophet MAE): {best_mae:.3f}")

if prophet_maes:
    avg_mae = sum(prophet_maes.values()) / len(prophet_maes)
    avg_mape = sum(prophet_mapes.values()) / len(prophet_mapes)
    print(f"Average Prophet MAE: {avg_mae:.3f}")

# prophet_details now holds per-series tuning information with expanded hyperparameter search

Seasonal+Trend (Prophet MAE): 1.907
Linear Trend (Prophet MAE): 2.228
Linear Trend (Prophet MAE): 2.228
Quadratic (Prophet MAE): 12.980
Quadratic (Prophet MAE): 12.980
Logistic (S-curve) (Prophet MAE): 2.874
Logistic (S-curve) (Prophet MAE): 2.874
Random Walk (drift) (Prophet MAE): 1.662
Random Walk (drift) (Prophet MAE): 1.662
Piecewise (changepoints) (Prophet MAE): 1.911
Piecewise (changepoints) (Prophet MAE): 1.911
Spiky Intermittent (Prophet MAE): 1.648
Spiky Intermittent (Prophet MAE): 1.648
Multi-seasonal (Prophet MAE): 1.404
Multi-seasonal (Prophet MAE): 1.404
Sunspots: Prophet failed; skipping.
Sunspots: Prophet failed; skipping.
CO2 (Prophet MAE): 0.365
CO2 (Prophet MAE): 0.365
CPIAUCSL (Prophet MAE): 1.494
CPIAUCSL (Prophet MAE): 1.494
UNRATE (Prophet MAE): 0.256
UNRATE (Prophet MAE): 0.256
INDPRO (Prophet MAE): 0.868
Average Prophet MAE: 2.466
INDPRO (Prophet MAE): 0.868
Average Prophet MAE: 2.466


### SKtime Performance

In [127]:
# Simplified SKtime Performance (base sktime only) with full silent execution wrapper
import logging, warnings, contextlib, io
import numpy as np
import pandas as pd

from sktime.forecasting.base import ForecastingHorizon
from sktime.performance_metrics.forecasting import mean_absolute_error as sk_mae
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.compose import EnsembleForecaster, make_reduction
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.ets import AutoETS
from sktime.transformations.series.detrend import Deseasonalizer, Detrender

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
try:
    from xgboost import XGBRegressor
    _HAS_XGB = True
except Exception:
    _HAS_XGB = False


def _simple_split(df: pd.DataFrame, holdout: int):
    df = df.sort_values('date')
    y = pd.Series(df['value'].values, index=pd.to_datetime(df['date']))
    if len(y) <= holdout + 5:
        raise ValueError("Series too short")
    return y.iloc[:-holdout], y.iloc[-holdout:]


def _make_es_grid():
    """Create a small, fast grid of ExponentialSmoothing models.

    We keep this intentionally compact to avoid blowing up runtime.
    Adjust `es_trends`, `es_seasonals`, `es_sp`, `damped_opts` to explore more.
    """
    es_trends = ['add', None]              # try additive trend & no trend
    es_seasonals = ['add', 'mul', None]    # additive, multiplicative, or none
    es_sp = [12]                           # monthly-ish seasonality (adjust as needed)
    damped_opts = [False, True]            # only meaningful if trend present

    models = []
    for tr in es_trends:
        for seas in es_seasonals:
            # If no seasonality, we don't actually need sp; but sktime still wants an int >=1
            for sp in (es_sp if seas is not None else [1]):
                for damp in damped_opts:
                    if tr is None and damp:  # damped only valid with a trend component
                        continue
                    label = (
                        f"ETS-{tr or 'none'}-{seas or 'none'}"
                        f"{'-damped' if damp else ''}-sp{sp if seas else 1}"
                    )
                    try:
                        mdl = ExponentialSmoothing(
                            trend=tr,
                            damped_trend=damp if tr is not None else False,
                            seasonal=seas,
                            sp=sp,
                        )
                        models.append((label, mdl))
                    except Exception:
                        # Skip any invalid combo silently
                        continue
    return models


def run_sktime_benchmark_silent(datasets, holdout=24, ensemble_top=3, silent=True):
    lines = []
    result_mae = {}
    best_model_map = {}
    deseason_map = {}
    detrend_map = {}
    # Capture / suppress warnings + stdout/stderr
    if silent:
        outer_stdout = io.StringIO()
        outer_stderr = io.StringIO()
        warn_ctx = warnings.catch_warnings()
        warn_ctx.__enter__()
        warnings.simplefilter("ignore")  # blanket ignore all warnings inside block
        out_redirect = contextlib.redirect_stdout(outer_stdout)
        err_redirect = contextlib.redirect_stderr(outer_stderr)
        out_redirect.__enter__()
        err_redirect.__enter__()
    try:
        for name, df in datasets.items():
            if len(df) <= holdout + 25:
                continue
            try:
                y_tr, y_ho = _simple_split(df, holdout)
            except Exception:
                continue
            if len(y_tr) < 30:
                continue
            fh = ForecastingHorizon(y_ho.index, is_relative=False)

            # === Deseasonalize & Detrend (minimal addition) ===
            y_tr_model = y_tr
            deseason = None
            detrender = None
            deseason_reason = ""
            detrend_reason = ""
            try:
                if len(y_tr) >= 12:
                    try:
                        deseason = Deseasonalizer()
                        y_tr_model = deseason.fit_transform(y_tr_model)
                        deseason_reason = "applied"
                    except Exception:
                        deseason = None
                        deseason_reason = "fail"
                else:
                    deseason_reason = "short"
                try:
                    detrender = Detrender(forecaster=PolynomialTrendForecaster(degree=1))
                    y_tr_model = detrender.fit_transform(y_tr_model)
                    detrend_reason = "applied"
                except Exception:
                    detrender = None
                    detrend_reason = "fail"
            except Exception:
                y_tr_model = y_tr
                deseason = None
                detrender = None
                if not deseason_reason:
                    deseason_reason = "fail"
                if not detrend_reason:
                    detrend_reason = "fail"

            # Heuristic seasonal period guess for seasonal naive
            sp_guess = 12 if len(y_tr) >= 24 else 1
            try:
                inf_freq = pd.infer_freq(y_tr.index)
                if inf_freq:
                    if inf_freq.startswith('W'):
                        # weekly data
                        sp_guess = 52 if len(y_tr) >= 104 else max(2, min(26, len(y_tr)//2))
                    elif inf_freq[0] == 'M':
                        sp_guess = 12
                    elif inf_freq.startswith('Q'):
                        sp_guess = 4
                    elif inf_freq[0] == 'D':
                        sp_guess = 7 if len(y_tr) >= 14 else 1
                if sp_guess > len(y_tr)//2:
                    sp_guess = max(1, len(y_tr)//4)
            except Exception:
                sp_guess = 12 if len(y_tr) >= 24 else 1
            if sp_guess < 1:
                sp_guess = 1

            # Base (non-ES) models + Seasonal Naive
            base_models = [
                ("Naive-last", NaiveForecaster(strategy="last")),
                ("Naive-drift", NaiveForecaster(strategy="drift")),
                ("Naive-seasonal", NaiveForecaster(strategy="seasonal_last", sp=sp_guess)),  # seasonal naive
                ("Theta", ThetaForecaster(sp=12)),  # sp=12 seasonal period
                ("Trend1", PolynomialTrendForecaster(degree=1)),
                ("Trend2", PolynomialTrendForecaster(degree=2)),
                ("AutoARIMA", AutoARIMA(suppress_warnings=True, maxiter=25)),
                ("AutoETS", AutoETS(auto=True)),
            ]

            # Add ES hyperparameter grid models
            es_models = _make_es_grid()

            # Add Reduction models (tree/boost regressors)
            reduction_models = []
            try:
                # Window lengths scale with series length but capped
                rf_win = min(48, max(6, len(y_tr)//10))
                reduction_models.append(
                    ("RF-Reduction", make_reduction(
                        RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1),
                        strategy="recursive",
                        window_length=rf_win,
                    ))
                )
                gb_win = min(36, max(6, len(y_tr)//12))
                reduction_models.append(
                    ("GB-Reduction", make_reduction(
                        GradientBoostingRegressor(random_state=42),
                        strategy="recursive",
                        window_length=gb_win,
                    ))
                )
                if _HAS_XGB:
                    xgb_win = min(48, max(6, len(y_tr)//10))
                    reduction_models.append(
                        ("XGB-Reduction", make_reduction(
                            XGBRegressor(
                                n_estimators=400,
                                max_depth=4,
                                learning_rate=0.05,
                                subsample=0.9,
                                colsample_bytree=0.8,
                                objective='reg:squarederror',
                                random_state=42,
                                n_jobs=-1,
                                verbosity=0,
                            ),
                            strategy="recursive",
                            window_length=xgb_win,
                        ))
                    )
            except Exception:
                pass

            models = base_models + es_models + reduction_models

            scores = []
            for lbl, mdl in models:
                try:
                    mdl.fit(y_tr_model)
                    pred = mdl.predict(fh)
                    # Inverse transforms (trend first, then season) if applied
                    try:
                        if detrender is not None:
                            pred = detrender.inverse_transform(pred)
                        if deseason is not None:
                            pred = deseason.inverse_transform(pred)
                    except Exception:
                        pass  # if inversion fails, keep current pred
                    mae = sk_mae(y_ho, pred)
                    if np.isfinite(mae):
                        scores.append((mae, lbl, mdl))
                except Exception:
                    continue
            if not scores:
                continue
            scores.sort(key=lambda x: x[0])
            best_mae, best_lbl, _ = scores[0]

            # Optional simple ensemble (ensemble operates on refit original transforms for consistency)
            if ensemble_top and ensemble_top > 1 and len(scores) >= ensemble_top:
                try:
                    topk = scores[:ensemble_top]
                    ens = EnsembleForecaster([(lbl, mdl) for _, lbl, mdl in topk])
                    ens.fit(y_tr_model)
                    e_pred = ens.predict(fh)
                    try:
                        if detrender is not None:
                            e_pred = detrender.inverse_transform(e_pred)
                        if deseason is not None:
                            e_pred = deseason.inverse_transform(e_pred)
                    except Exception:
                        pass
                    e_mae = sk_mae(y_ho, e_pred)
                    if e_mae < best_mae:
                        best_mae = e_mae
                        best_lbl = f"EnsembleTop{ensemble_top}"
                except Exception:
                    pass

            result_mae[name] = float(best_mae)
            best_model_map[name] = best_lbl
            deseason_map[name] = deseason_reason or 'none'
            detrend_map[name] = detrend_reason or 'none'
            # Append transform status info
            lines.append(
                f"{name}: {best_mae:.3f} (sktime {best_lbl}) [deseason={deseason_map[name]} detrend={detrend_map[name]}]"
            )
    finally:
        if silent:
            # Exit contexts silently; discard captured text
            err_redirect.__exit__(None, None, None)
            out_redirect.__exit__(None, None, None)
            warn_ctx.__exit__(None, None, None)

    df_results = (
        pd.DataFrame({
            'dataset': list(result_mae.keys()),
            'sktime_MAE': [result_mae[k] for k in result_mae.keys()],
            'best_model': [best_model_map[k] for k in result_mae.keys()],
            'deseason_status': [deseason_map[k] for k in result_mae.keys()],
            'detrend_status': [detrend_map[k] for k in result_mae.keys()],
        })
        .sort_values('sktime_MAE').reset_index(drop=True)
    )
    return {'lines': lines, 'mae': result_mae, 'frame': df_results, 'best_model_map': best_model_map}

# Run benchmark silently
_sktime_out = run_sktime_benchmark_silent(datasets, holdout=24, ensemble_top=3, silent=True)
sktime_results_df = _sktime_out['frame']

In [128]:
((5.864*10) - 42.475)/9

1.796111111111111

In [131]:
avg_mae = (sktime_results_df['sktime_MAE'].to_numpy().sum() - 26.994923)/12

avg_mae

1.6978286262648485

In [130]:
sktime_results_df

,dataset,sktime_MAE,best_model,deseason_status,detrend_status
0,UNRATE,0.176712,EnsembleTop3,fail,applied
1,CO2,0.286098,XGB-Reduction,fail,applied
2,INDPRO,1.337847,Naive-last,fail,applied
3,Quadratic,1.437959,Trend2,fail,applied
4,Random Walk (drift),1.479566,ETS-none-add-sp12,fail,applied
5,Seasonal+Trend,1.566502,XGB-Reduction,fail,applied
6,Multi-seasonal,1.569462,RF-Reduction,fail,applied
7,Spiky Intermittent,1.655270,ETS-add-add-damped-sp12,fail,applied
8,Linear Trend,2.145040,EnsembleTop3,fail,applied
9,Logistic (S-curve),2.324673,ETS-add-none-sp1,fail,applied


### XGB forecasting block

In [ ]:
import pandas as pd 
from typing import Optional, Tuple

def ts_train_test_split(
    X: pd.DataFrame, 
    y: pd.Series, 
    outcome_col: str, 
    date_col: str, 
    fdw: int, 
    holdout_window: int,
    forecast_window: Optional[int] = 1 
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
    """
    Apply preprocessing and split the data into training and testing sets for time series modeling.
    """

    # Helper function to preprocess ts data
    def _ts_preproc(inp_tbl: pd.DataFrame, inp_y: pd.Series) -> Tuple[pd.DataFrame, pd.Series]:   
        preproc_tbl = (inp_tbl
        .pipe(lambda x: x.assign(**{f"lagged_{outcome_col}_{i}m": x[outcome_col].shift(i) for i in range(forecast_window, fdw + 1)}))
        .pipe(lambda x: x.assign(**{f"inv_hyp_sin_lagged_{outcome_col}_{i}m": np.arcsinh(x[outcome_col].shift(i)) for i in range(forecast_window, fdw + 1)}))
        .pipe(lambda x: x.assign(**{f"rolling_avg_{outcome_col}_{i}m": x[outcome_col].shift(1).rolling(window=i).mean() for i in range(forecast_window, fdw + 1)}))
        .pipe(lambda x: x.assign(**{f"min_{outcome_col}_{i}m": x[outcome_col].shift(1).rolling(window=i).min() for i in range(forecast_window, fdw + 1)}))
        # New time and seasonal features
        .pipe(lambda x: x.assign(
            # t=np.arange(len(x)),
            monthsin=np.sin(2 * np.pi * pd.to_datetime(x[date_col]).dt.month / 12.0),
            monthcos=np.cos(2 * np.pi * pd.to_datetime(x[date_col]).dt.month / 12.0),
        ))
        # Drop the original date and outcome columns
        .drop([date_col, outcome_col], axis=1)
        # Rowwise deletion of missing values
        .dropna(axis=0)
        )
        preproc_y = inp_y.loc[preproc_tbl.index]

        return preproc_tbl, preproc_y

    # Reset index of X and y
    X.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)

    # Calculate the index to split the data
    train_end_index = X.shape[0] - (holdout_window)
    test_start_index = X.shape[0] - (fdw + holdout_window)

    # Split the data
    X_train = X.iloc[:train_end_index]
    X_test = X.iloc[test_start_index:]
    y_train = y.iloc[:train_end_index]
    y_test = y.iloc[test_start_index:]

    # Set the indices of both X and y train/test to the 'date' column 
    X_train.set_index(date_col, drop=False, inplace=True)
    y_train.index = X_train.index
    X_test.set_index(date_col, drop=False, inplace=True)
    y_test.index = X_test.index

    # Preprocess the data
    X_train, y_train = _ts_preproc(X_train, y_train)
    X_test, y_test = _ts_preproc(X_test, y_test)

    return X_train, X_test, y_train, y_test


In [ ]:
from xgboost import XGBRegressor
from automl_tool.estimation import XGBWithEarlyStoppingRegressor
from pandas_datareader import data as pdr
from automl_tool.preprocessing import ts_train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np 

fdw, holdout_window, forecast_window = 12, 24, 1

df_fred = pdr.DataReader("CPIAUCSL", 'fred', start='1990-01-01')
df_fred = df_fred.rename(columns={'CPIAUCSL': 'value'}).reset_index().rename(columns={'DATE': 'date'})
df_fred = df_fred.dropna()

X, y = df_fred, df_fred['value']
X_train, X_holdout, y_train, y_holdout = ts_train_test_split(
	X, y, 'value', 'date', fdw, holdout_window, forecast_window=forecast_window
)

xgb_model = XGBWithEarlyStoppingRegressor()
xgb_model.fit(X_train, y_train)

preds = xgb_model.predict(X_holdout)
mae = mean_absolute_error(y_holdout, preds)
print(f"XGBRegressor MAE on CPIAUCSL: {mae:.3f}")


In [ ]:
xgb_model

#### AutoML forecasting block

In [ ]:
from automl_tool.automl import AutoML

automl_mod = AutoML(X_train, y_train, 'value', time_series=True)

automl_mod.fit_pipeline(holdout_window=holdout_window)

# Get the best model from the fitted pipeline
y_preds = automl_mod.fitted_pipeline.predict(X_holdout)

mean_absolute_error(y_holdout, y_preds)


In [ ]:

# Plot actual and predicts on holdout set 
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(y_holdout.index, y_holdout, label='Actual', color='blue')
plt.plot(y_holdout.index, y_preds, label='Predicted', color='orange')
legend = plt.legend(loc='upper left', fontsize=12)
plt.title('Predictions vs Actuals on Holdout Set')


In [ ]:
automl_mod.fitted_pipeline

### Scalecast forecasting block


In [ ]:

import pandas as pd
from scalecast.Forecaster import Forecaster
from scalecast import GridGenerator

GridGenerator.get_example_grids()  # example hyperparameter grids

data = df_fred
f = Forecaster(
    y=data['value'],               # required
    current_dates=data['date'],    # required
    future_dates=1,               # length of the forecast horizon
    test_length=24,                 # set a test set length or fraction to validate all models if desired
    cis=False,                     # choose whether or not to evaluate confidence intervals for all models
)
f.set_estimator('xgboost')  # select an estimator

f.auto_Xvar_select()       # find best look-back, trend, and seasonality for your series
f.cross_validate(k=3)       # tune model hyperparams using time series cross validation
f.auto_forecast()           # automatically forecast with the chosen Xvars and hyperparams

results = f.export(['lvl_fcsts','model_summaries'])


In [ ]:
ts_preds = f.export('lvl_test_set_predictions')

mean_absolute_error(ts_preds['actual'], ts_preds['xgboost'])

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_holdout.index, ts_preds['actual'], label='Actual', color='blue')
plt.plot(y_holdout.index, ts_preds['xgboost'], label='Predicted', color='orange')
legend = plt.legend(loc='upper left', fontsize=12)
plt.title('XGBWithEarlyStoppingRegressor Predictions vs Actuals on Holdout Set')

In [ ]:
from scalecast import GridGenerator
GridGenerator.get_example_grids()   # writes Grids.py to your working dir (if not already present)

# then either open Grids.py in your editor, or import it:
from Grids import xgboost as xgb_grid
print(xgb_grid)


In [ ]:
results['model_summaries']['HyperParams'][0]